# 4. 크롤링 이해하기

API를 이용하면 데이터를 매우 쉽게 수집할 수 있지만, 국내 주식 데이터를 다운로드 하기에는 한계가 있으며, 원하는 데이터가 API의 형태로 제공된다는 보장도 없습니다. 따라서 우리는 필요한 데이터를 얻기 위해 직접 찾아 나서야 합니다.

각종 금융 웹사이트에는 주가, 재무정보 등 우리가 원하는 대부분의 주식 정보가 제공되고 있으며, API를 활용할 수 없는 경우에도 크롤링을 통해 이러한 데이터를 수집할 수 있습니다. 크롤링 혹은 스크래핑이란 웹사이트에서 원하는 정보를 수집하는 기술입니다. 대부분의 금융 웹사이트는 간단한 형태로 작성되어 있어, 몇 가지 기술만 익히면 어렵지 않게 데이터를 크롤링할 수 있습니다. 이번 장에서는 크롤링에 대한 간단한 설명과 예제를 살펴보겠습니다.

```{note}
크롤링을 할 때 주의해야 할 점이 있습니다. 특정 웹사이트의 페이지를 쉬지 않고 크롤링하는 행위를 무한 크롤링이라고 합니다. 무한 크롤링은 해당 웹사이트의 자원을 독점하게 되어 타인의 사용을 막게 되며 웹사이트에 부하를 주게 됩니다. 일부 웹사이트에서는 동일한 IP로 쉬지 않고 크롤링을 할 경우 접속을 막아버리는 경우도 있습니다. 따라서 하나의 페이지를 크롤링한 후 1~2초 가량 정지하고 다시 다음 페이지를 크롤링하는 것이 좋습니다.

또한 신문기사나 책, 논문, 사진 등 저작권이 있는 자료를 통해 부당이득을 얻는다는 등의 행위를 할 경우 법적 제재를 받을 수 있습니다. 

이 책에서 설명하는 크롤링을 통해, 상업적 가치가 있는 데이터에 접근을 시도하여 발생할 수 있는 어떠한 상황에 대해서도 책임을 질 수 없다는 점을 명심하시기 바랍니다.
```

## 4.1 GET과 POST 방식 이해하기

우리가 인터넷에 접속해 서버에 파일을 요청(Request)하면, 서버는 이에 해당하는 파일을 우리에게 보내줍니다(Response). 크롬과 같은 웹 브라우저는 이러한 과정을 사람이 수행하기 편하고 시각적으로 보기 편하도록 만들어진 것이며, 인터넷 주소는 서버의 주소를 기억하기 쉽게 만든 것입니다. 우리가 서버에 데이터를 요청하는 형태는 다양하지만 크롤링에서는 주로 GET과 POST 방식을 사용합니다.

```{figure} image/crawl_flow.png
---
name: crawl_flow
---
클라이언트와 서버 간의 요청/응답 과정
```

### 4.1.1 GET 방식

GET 방식은 인터넷 주소를 기준으로 이에 해당하는 데이터나 파일을 요청하는 것입니다. 주로 클라이언트가 요청하는 쿼리를 앰퍼샌드(&) 혹은 물음표(?) 형식으로 결합해 서버에 전달합니다.

네이버 홈페이지에 접속한 후 [퀀트]를 검색하면, 주소 끝부분에 [&query=퀀트]가 추가되며 이에 해당하는 페이지의 내용을 보여줍니다. 즉, 해당 페이지는 GET 방식을 사용하고 있으며 입력 종류는 query, 입력값은 퀀트임을 알 수 있습니다.

```{figure} image/04_naver_search1.png
---
name: 04_naver_search1
---
네이버 검색 결과
```

이번에는 [헤지펀드]를 검색하면, 주소 끝부분이 [&query=헤지펀드&oquery=퀀트...]로 변경됩니다. 현재 입력값은 헤지펀드, 기존 입력값은 퀀트이며 이러한 과정을 통해 연관검색어가 생성됨도 유추해볼 수 있습니다.

```{figure} image/04_naver_search2.png
---
name: 04_naver_search2
---
네이버 재검색 결과
```

### 4.1.2 POST 방식

POST 방식은 사용자가 필요한 값을 추가해서 요청하는 방법입니다. GET 방식과 달리 클라이언트가 요청하는 쿼리를 body에 넣어서 전송하므로 요청 내역을 직접 볼 수 없습니다.

동행복권 홈페이지에 접속해 [당첨결과] 메뉴를 확인해보겠습니다. 

```
https://www.dhlottery.co.kr/gameResult.do?method=byWin
```

```{figure} image/crawl_lotto.png
---
name: crawl_lotto
---
회차별 당첨번호
```

이번엔 회차 바로가기를 변경한 후 [조회]를 클릭합니다. 페이지의 내용은 선택일 기준으로 변경되었지만, 주소는 변경되지 않고 그대로 남아 있습니다. GET 방식에서는 입력 항목에 따라 웹페이지 주소가 변경되었지만, POST 방식을 사용해 서버에 데이터를 요청하는 해당 웹사이트는 그렇지 않은 것을 알 수 있습니다.

POST 방식의 데이터 요청 과정을 살펴보려면 개발자 도구를 이용해야 하며, 크롬에서는 [F12]키를 눌러 개발자 도구 화면을 열 수 있습니다. 개발자 도구 화면을 연 상태에서 다시 한번 [조회]를 클릭해봅니다. [Network] 탭을 클릭하면, [조회]을 클릭함과 동시에 브라우저와 서버 간의 통신 과정을 살펴볼 수 있습니다. 이 중 상단의 gameResult.do?method=byWin 이라는 항목이 POST 형태임을 알 수 있습니다.

```{figure} image/crawl_lotto_post.png
---
name: crawl_lotto_post
---
크롬 개발자도구의 Network 화면
```

해당 메뉴를 클릭하면 통신 과정을 좀 더 자세히 알 수 있습니다. 가장 하단의 Form Data에는 서버에 데이터를 요청하는 내역이 있습니다. drwNo에는와 dwrNoList에는 선택한 회차의 숫자가 들어가있습니다.

```{figure} image/crawl_lotto_query.png
---
name: crawl_lotto_query
---
POST 방식의 서버 요청 내역
```

이처럼 POST 방식은 요청하는 데이터에 대한 쿼리가 GET 방식처럼 URL을 통해 전송되는 것이 아닌 body를 통해 전송되므로, 이에 대한 정보는 웹 브라우저를 통해 확인할 수 없습니다

## 4.2 크롤링 예제

일반적으로 크롤링은 {numref}`crawl_flowchart`의 과정을 따릅니다. 먼저, request 패키지의 `get()` 혹은 `post()` 함수를 이용해 데이터를 다운로드한 후 bs4 패키지의 함수들을 이용해 원하는 데이터를 찾는 과정으로 이루어집니다. 기본적인 크롤링을 시작으로 GET 방식과 POST 방식으로 데이터를 받는 예제를 학습해 보겠습니다.

```{figure} image/crawl_flowchart.png
---
name: crawl_flowchart
---
일반적인 크롤링 과정
```

### 4.2.1 명언 크롤링하기

크롤링의 간단한 예제로 'Quotes to Scrape' 사이트에 있는 명언을 수집하겠습니다.

```
https://quotes.toscrape.com/
```

먼저 해당사이트에 접속한 후, 명언에 해당하는 부분에 마우스 커서를 올려둔 후 마우스 오른쪽 버튼을 클릭하고 [검사]를 선택하면 개발자 도구 화면이 나타납니다. 여기서 해당 글자가 HTML 내에서 어떤 부분에 위치하는지 확인할 수 있습니다. 각 네모에 해당하는 부분은 [div 태그의 quote 클래스]에 위치하고 있으며, 명언은 [div 태그 → span 태그의 text 클래스]에, 말한 사람은 [div 태그 → span 태그 → small 태그의 author 클래스]에 위치하고 있습니다.

```{figure} image/crawl_quote.png
---
name: crawl_quote
---
Quotes to Scrape의 명언부분 HTML
```

먼저 해당 페이지의 내용을 불러옵니다.

In [1]:
import requests as rq

url = 'https://quotes.toscrape.com/'
quote = rq.get(url)

print(quote)

<Response [200]>


먼저 url에 해당 주소를 입력한 후 `get()` 함수를 이용해 해당 페이지의 내용을 받았습니다. 이를 확인해보면 Response가 200, 즉 데이터가 이상 없이 받아졌음이 확인됩니다. 

In [2]:
quote.content[:1000]

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n\t<meta charset="UTF-8">\n\t<title>Quotes to Scrape</title>\n    <link rel="stylesheet" href="/static/bootstrap.min.css">\n    <link rel="stylesheet" href="/static/main.css">\n</head>\n<body>\n    <div class="container">\n        <div class="row header-box">\n            <div class="col-md-8">\n                <h1>\n                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>\n                </h1>\n            </div>\n            <div class="col-md-4">\n                <p>\n                \n                    <a href="/login">Login</a>\n                \n                </p>\n            </div>\n        </div>\n    \n\n<div class="row">\n    <div class="col-md-8">\n\n    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">\n        <span class="text" itemprop="text">\xe2\x80\x9cThe world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.\xe2\x80\

content를 통해 함수를 통해 받아온 내용을 확인할 수 있지만, 텍스트 형태로 되어있습니다. `BeautifulSoup()` 함수를 이용해 원하는 HTML 요소에 접근하기 쉬운 형태로 변경합니다.

In [3]:
from bs4 import BeautifulSoup

quote_html = BeautifulSoup(quote.content, 'html.parser')
quote_html.head()

[<meta charset="utf-8"/>,
 <title>Quotes to Scrape</title>,
 <link href="/static/bootstrap.min.css" rel="stylesheet"/>,
 <link href="/static/main.css" rel="stylesheet"/>]

`BeautifulSoup()` 함수 내에 HTML 정보에 해당하는 quote.content와 파싱 방법에 해당하는 html.parser를 입력하면 개발자 도구 화면에서 보던 것과 비슷한 형태로 변경되며, 이를 통해 원하는 요소의 데이터를 읽어올 수 있습니다.

우리는 개발자 도구 화면에서 명언에 해당하는 부분이 [div 태그의 quote 클래스 → span 태그의 text 클래스]에 위치하고 있음을 살펴보았습니다. 이를 활용해 명언만을 추출하는 방법은 다음과 같습니다.

In [4]:
quote_div = quote_html.find_all('div', {'class' : 'quote'})
quote_div[0]

<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>

`find_all()` 함수를 이용할 경우 원하는 태그의 값들을 찾아올 수 있습니다. 먼저 태그에 해당하는 div를 입력하고, class에 해당하는 quote는 딕셔너리 형태로 입력합니다. 해당 내역을 출력해보면 원하는 내용을 찾아왔으며, 이제 여기서 span 태그의 text 클래스에 해당하는 내용을 추가로 찾겠습니다.

In [5]:
quote_div[0].find_all('span', {'class' : 'text'})[0].text

'“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'

다시 한번 `find_all()` 함수를 이용해 원하는 부분을 입력하면 우리가 원하던 명언에 해당하는 내용이 찾아지며, `.text`를 통해 텍스트 데이터만을 출력할 수 있습니다. `for loop` 구문을 이용하면 명언에 해당하는 부분을 한번에 추출할 수 있습니다.

In [6]:
quote_div = quote_html.find_all('div', {'class' : 'quote'})

for i in quote_div:
    print(i.find_all('span', {'class' : 'text'})[0].text)

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”


위 예제에서는 간단하게 원하는 데이터를 찾았지만, 태그를 여러번 찾아 내려가야 할 경우 `find_all()` 함수를 이용하는 방법은 매우 번거롭습니다. `select()` 함수의 경우 좀더 쉬운 방법으로 원하는 데이터를 찾을 수 있습니다.

In [7]:
quote_text = quote_html.select('div.quote > span.text')

quote_text

[<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>,
 <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>,
 <span class="text" itemprop="text">“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”</span>,
 <span class="text" itemprop="text">“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”</span>,
 <span class="text" itemprop="text">“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”</span>,
 <span class="text" itemprop="text">“Try not to become a man of success. Rather become a man of value.”</span>,
 <span class="text" itemprop="text">“It is better to be hated for what you are than to be loved for what you are not.

`select()` 함수는 찾고자 하는 태그를 입력하며, 클래스명이 존재할 경우 점(.)을 붙여줍니다. 또한 여러 태그를 찾아 내려가야할 경우 화살표를 이용해 순서대로 입력해주면 됩니다.

In [8]:
[i.text for i in quote_text]

['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”',
 '“It is our choices, Harry, that show what we truly are, far more than our abilities.”',
 '“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”',
 '“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”',
 "“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”",
 '“Try not to become a man of success. Rather become a man of value.”',
 '“It is better to be hated for what you are than to be loved for what you are not.”',
 "“I have not failed. I've just found 10,000 ways that won't work.”",
 "“A woman is like a tea bag; you never know how strong it is until it's in hot water.”",
 '“A day without sunshine is like, you know, night.”']

리스트 내에서 `for loop`를 이용할 경우 코드가 더욱 깔끔해집니다.

In [9]:
quote_author = quote_html.select('div.quote > span > small.author')

[i.text for i in quote_author]

['Albert Einstein',
 'J.K. Rowling',
 'Albert Einstein',
 'Jane Austen',
 'Marilyn Monroe',
 'Albert Einstein',
 'André Gide',
 'Thomas A. Edison',
 'Eleanor Roosevelt',
 'Steve Martin']

동일한 방법을 이용해 말한 사람 역시 손쉽게 추출이 가능합니다.

```{note}
`BeautifulSoup()` 함수 내 파싱 방법으로 html.parser를 이용했습니다. 해당 함수에는 이 외에도 다양한 파서를 지원하며, 그 내용은 다음과 같습니다.

| Parser | 선언방법 | 장점 | 단점 |
| --- | --- | --- | --- |
| html.parser | BeautifulSoup(내용, 'html.parser') | 설치할 필요 없음 <br> 적당한 속도 | 
| lxml HTML parser | BeautifulSoup(내용, 'lxml') | 매우 빠름 | lxml 추가 설치 필요 |
| lxml XML parser | BeautifulSoup(내용, 'xml') | 매우 빠름 <br> 유일하게 XML 파싱 | lxml 추가 설치 필요 |
| html5lib | BeautifulSoup(내용, 'html5lib') | 웹 브라우저와 같은 방식으로 페이지 파싱. <br> 유효한 HTML5 생성 | html5lib 추가 설치 필요 <br> 매우 느림 |
```

### 4.2.2 금융 속보 크롤링

이번에는 금융 속보의 제목을 추출해보겠습니다. 먼저 네이버 금융에 접속한 후 [뉴스 → 실시간 속보]를 선택합니다. 이 중 뉴스의 제목에 해당하는 텍스트만 추출하고자 합니다.

```
https://finance.naver.com/news/news_list.nhn?mode=LSS2D&section_id=101&section_id2=258
```

개발자 도구 화면을 통헤 제목에 해당하는 부분은 [dl 태그 → dd 태그의 articleSubject 클래스 → a 태그 중 title 속성]에 위치하고 있음을 확인할 수 있습니다. 태그와 속성의 차이가 이해되지 않은 분은 2장을 다시 살펴보기 바랍니다.

```{figure} image/crawl_naver_news.png
---
name: crawl_naver_news
---
실시간 속보의 제목 부분 HTML
```

먼저 해당 페이지의 내용을 불러옵니다.

In [10]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS2D&section_id=101&section_id2=258'
data = rq.get(url)

먼저 url에 해당 주소를 입력한 후 `get()` 함수를 이용해 해당 페이지의 내용을 받아 저장합니다.

우리는 개발자 도구 화면을 통해 제목에 해당하는 부분이 [dl 태그 → dd 태그의 articleSubject 클래스 → a 태그 중 title 속성]에 위치하고 있음을 살펴보았습니다. 이를 활용해 제목 부분만을 추출하는 방법은 다음과 같습니다.

In [11]:
html = BeautifulSoup(data.content, 'html.parser')
html_select = html.select('dl > dd.articleSubject > a')

html_select[0]

<a href="/news/news_read.nhn?article_id=0000595743&amp;office_id=469&amp;mode=LSS2D&amp;type=0§ion_id=101§ion_id2=258§ion_id3=&amp;date=20210409&amp;page=1" title='국민연금 주식매도 멈춘다...  "동학개미 압박에 국민노후 저당" 논란'>국민연금 주식매도 멈춘다...  "동학개미 압박에 국민노후 저당" 논란</a>

먼저 `BeautifulSoup()` 함수를 통해 HTML 정보를 불러옵니다. 그 후 `select()` 함수를 통해 원하는 태그로 접근해 들어갑니다. 이 중 우리가 원하는 최종 내용은 title 속성에 위치하고 있으므로, 추가적으로 이를 추출합니다.

In [12]:
html_select[0].get('title')

'국민연금 주식매도 멈춘다...\xa0 "동학개미 압박에 국민노후 저당" 논란'

`get()` 함수를 통해 손쉽게 속성에 해당하는 내용을 추출할 수 있습니다. `for loop` 구문으로 묶어 한번에 제목들을 추출하도록 하겠습니다.

In [13]:
html = BeautifulSoup(data.content, 'html.parser')
html_select = html.select('dl > dd.articleSubject > a')
[i.get('title') for i in html_select]

['국민연금 주식매도 멈춘다...\xa0 "동학개미 압박에 국민노후 저당" 논란',
 '[픽뉴스] 친환경 용기라더니, 무례한 터키대통령, 28년간 기른 손톱',
 '김주하 앵커가 전하는 4월 9일 종합뉴스 주요뉴스',
 '넥스턴 "계열사 넥스아이디랩 주식 100억원에 취득"',
 '넥스턴 "계열사 넥스턴바이오 주식 100억원에 취득"',
 '[표]아시아 주요 증시 동향(4월 9일)',
 '군인공제회, 에티오피아 6·25 참전용사에 기부금 전달',
 '[fn마켓워치]메테우스운용, 이천 복합물류센터 개발 추진',
 "국민연금, '자동 매도' 멈춘다…이제 코스피 매수할듯",
 '에르메스 첫 실적공개, 지난해 한국서 4200억 벌었다…영업익 1334억',
 '파미니티, 국회 표창장…"韓·亞 교류 협력 공로 인정받아"',
 '[일문일답] 국민연금 "추가 매입·매도 중단 아냐…수익성 본다"',
 '한진 1Q 영업익 133억, 전년比 47.6%↓…"택배 분류인력 비용 등 발생"']

### 4.2.3 기업공시채널에서 오늘의 공시 불러오기

한국거래소 상장공시시스템(kind.krx.co.kr)에 접속한 후 [오늘의 공시 → 전체 → 더보기]를 선택해 전체 공시내용을 확인할 수 있습니다.

```{figure} image/crawl_kind.png
---
name: crawl_kind
---
오늘의공시 확인하기
```

해당 페이지에서 날짜를 변경한 후 [검색]을 누르면, 페이지의 내용은 해당일의 공시로 변경되지만 URL은 변경되지 않습니다. 이처럼 POST 방식은 요청하는 데이터에 대한 쿼리가 body의 형태를 통해 전송되므로, 개발자 도구 화면을 통해 해당 쿼리에 대한 내용을 확인해야 합니다.

개발자 도구 화면을 연 상태에서 조회일자를 원하는 날짜로 선택, [검색]을 클릭한 후 [Network] 탭의 todaydisclosure.do 항목을 살펴보면 Form Data를 통해 서버에 데이터를 요청하는 내역을 확인할 수 있습니다. 여러 항목 중 selDate 부분이 우리가 선택한 일자로 설정되어 있습니다.

```{figure} image/crawl_kind_post.png
---
name: crawl_kind_post
---
POST 방식의 데이터 요청
```

POST 방식으로 쿼리를 요청하는 방법을 코드로 나타내면 다음과 같습니다.

In [14]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://kind.krx.co.kr/disclosure/todaydisclosure.do'
query = {
    'method': 'searchTodayDisclosureSub',
    'currentPageSize': '15',
    'pageIndex': '1',
    'orderMode': '0',
    'orderStat': 'D',
    'forward': 'todaydisclosure_sub',
    'chose': 'S',
    'todayFlag': 'N',
    'selDate': '2021-04-09'
}

data = rq.post(url, query)
data_bs = BeautifulSoup(data.content, 'html.parser')
pd.read_html(data_bs.prettify())[0].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,18:55,넥스턴,타법인주식및출자증권취득결정,넥스턴,공시차트 주가차트
1,18:54,넥스턴,타법인주식및출자증권취득결정,넥스턴,공시차트 주가차트
2,18:40,좋은사람들,소송등의제기(신주발행유지가처분),좋은사람들,공시차트 주가차트
3,18:28,씨씨에스,기준가산정 등에 관한 안내('20.06.26일자 공시사항 재안내),코스닥시장본부,공시차트 주가차트
4,18:27,지와이커머스,주권매매거래정지해제(상장폐지에 따른 정리매매 개시),코스닥시장본부,공시차트 주가차트


1. 먼저 url과 쿼리를 입력하며, 쿼리는 딕셔너리 형태로 입력해줍니다. url은 todaydisclosure.do 항목에서 General 부분의 Request URL에 해당하는 값을, 쿼리는 Form Data와 동일하게 입력해줍니다. 쿼리 중 marketType과 같이 값이 없는 항목은 입력하지 않아도 됩니다.
2. `POST()` 함수를 통해 해당 url에 원하는 쿼리를 요청합니다.
3. `BeautifulSoup()` 함수를 통해 해당 페이지의 HTML 내용을 읽어옵니다.
4. 테이블 형태를 읽어오기 위해 `prettify()` 함수를 이용해 BeautifulSoup 에서 파싱한 파서 트리를 유니코드 형태로 변형해줍니다. 그 후, `read_html()` 함수를 통해 테이블을 데이터프레임 형태로 읽어옵니다.

데이터를 확인하면 화면과 동일한 내용이 출력됩니다. POST 형식의 경우 쿼리 내용을 바꾸어 원하는 데이터를 받을 수 있습니다. 만일 다른 날짜의 공시를 확인하고자 한다면 위의 코드에서 selDate만 해당일로 변경해주면 됩니다.

### 4.2.4 네이버 금융에서 주식티커 크롤링

태그와 속성, 페이지 내비게이션 값을 결합해 국내 상장주식의 종목명 및 티커를 추출하는 방법을 알아보겠습니다. 네이버 금융에서 [국내증시 → 시가총액] 페이지에는 코스피와 코스닥의 시가총액별 정보가 나타나 있습니다.

- 코스피: https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page=1
- 코스닥: https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page=1

또한 종목명을 클릭해 이동하는 페이지의 URL을 확인해보면, 끝 6자리가 각 종목의 거래소 티커임도 확인이 됩니다.

```
삼성전자: https://finance.naver.com/item/main.nhn?code=005930
```

티커 정리를 위해 HTML에서 확인해야 할 부분은 총 두 가지입니다. 먼저 하단의 페이지 내비게이션을 통해 코스피와 코스닥 시가총액에 해당하는 페이지가 각각 몇 번째 페이지까지 있는지 알아야 합니다. 아래와 같은 항목 중 [맨뒤]에 해당하는 페이지가 가장 마지막 페이지입니다.

```{figure} image/crawl_page_navi.png
---
name: crawl_page_navi
---
페이지 내비게이션
```
[맨뒤]에 마우스 커서를 올려두고 마우스 오른쪽 버튼을 클릭한 후 [검사]를 선택하면 개발자 도구 화면이 열립니다. 여기서 해당 글자가 HTML 내에서 어떤 부분에 위치하는지 확인할 수 있습니다. 해당 링크는 [td 태그 중 pgRR 클래스 → a 태그 중 href 속성]에 위치하며, page= 뒷부분의 숫자에 위치하는 페이지로 링크가 걸려 있습니다.

```{figure} image/crawl_page_navi2.png
---
name: crawl_page_navi2
---
HTML 내 페이지 내비게이션 부분
```

종목명 링크에 해당하는 주소 중 끝 6자리는 티커에 해당합니다. 따라서 각 링크들의 주소를 알아야 할 필요도 있습니다.

```{figure} image/crawl_naver_corp.png
---
name: crawl_naver_corp
---
종목 별 링크 주소 확인
```

삼성전자에 마우스 커서를 올려둔 후 마우스 오른쪽 버튼을 클릭하고 [검사]를 선택합니다. 개발자 도구 화면을 살펴보면 해당 링크가 [tbody → tr → td → a 태그의 href 속성]에 위치하고 있음을 알 수 있습니다.

위 정보들을 이용해 데이터를 다운로드하겠습니다. 아래 코드에서 i = 0일 경우 코스피에 해당하는 URL이 생성되고, i = 1일 경우 코스닥에 해당하는 URL이 생성됩니다. 먼저 코스피에 해당하는 데이터를 다운로드하겠습니다.

In [15]:
import requests as rq
from bs4 import BeautifulSoup

i = 0
ticker = {}
url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='+str(i)+'&page=1'
down_table = rq.get(url)

1. 빈 딕셔너리인 ticker 변수를 만들어줍니다.
2. 코스피 시가총액 페이지의 url을 만듭니다.
3. `get()` 함수를 통해 해당 페이지 내용을 받아 저장합니다.

가장 먼저 해야 할 작업은 마지막 페이지가 몇 번째 페이지인지 찾아내는 작업입니다. 우리는 이미 개발자 도구 화면을 통해 해당 정보가 어디에 위치하고 있음을 알고 있습니다.

In [16]:
down_html = BeautifulSoup(down_table.content, 'html.parser')
navi_final = down_html.select_one('td.pgRR > a').get('href')

print(navi_final)

/sise/sise_market_sum.nhn?sosok=0&page=32


1. `BeautifulSoup()` 함수를 이용해 해당 페이지의 HTML 내용을 읽어옵니다.
2. `select_one()` 함수 내에 찾고자 하는 태그를 입력합니다. `select()` 함수와 다르게 `select_one()` 함수는 첫번째 결과만을 반환합니다. 그 후 `get()` 함수를 통해 href 속성을 찾습니다.

이를 통해 navi_final에는 해당 부분에 해당하는 내용이 저장됩니다. 이 중 우리가 알고 싶은 내용은 page= 뒤에 있는 숫자입니다. 해당 내용을 추출하는 코드는 다음과 같습니다.

In [17]:
navi_final_number = navi_final.split('=')[-1]
navi_final_number = int(navi_final_number)

navi_final_number

32

1. `split()` 함수를 통해 =를 기준으로 문장을 나눠줍니다. 그 후 뒤에서 첫 번째 데이터만 선택합니다.
2. `int()` 함수를 통해 숫자 형태로 바꾸어 줍니다.

이를 통해 코스피 시가총액 페이지가 총 페이지까지 있는지 알 수 있으며, `for loop` 구문을 이용하면 1페이지부터 마지막 페이지까지 모든 내용을 읽어올 수 있습니다. 먼저 코스피의 첫 번째 페이지에서 우리가 원하는 데이터를 추출하는 방법을 살펴보겠습니다.

In [18]:
import pandas as pd

i = 0 # 코스피
j = 1 # 첫번째 페이지
url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='+str(i)+"&page="+str(j)

read_table = pd.read_html(url, encoding = "EUC-KR")
table = read_table[1]

1. pandas 패키지의 `read_html()` 함수를 이용하면 손쉽게 테이블 형태를 크롤링 할 수 있으며, 네이버의 경우 인코딩이 'EUC-KR' 형태이므로 이를 설정해줍니다.
2. table 변수에는 리스트 형태로 총 세 가지 테이블이 저장되어 있습니다. 첫 번째 리스트에는 거래량, 시가, 고가 등 적용 항목이 저장되어 있고 세 번째 리스트에는 페이지 내비게이션 테이블이 저장되어 있으므로, 우리에게 필요한 두 번째 리스트만을 table 변수에 저장합니다.

저장된 테이블 내용을 확인하면 다음과 같습니다.

In [19]:
table.head()

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE,토론실
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,삼성전자,83600.0,1100.0,-1.30%,100.0,4990738.0,5969783.0,54.83,17791122.0,21.77,9.99,NaN
2,2.0,SK하이닉스,140000.0,4000.0,-2.78%,5000.0,1019203.0,728002.0,50.53,3247389.0,21.43,9.53,NaN
3,3.0,NAVER,383500.0,2000.0,+0.52%,100.0,629950.0,164263.0,57.07,463512.0,62.90,15.22,NaN
4,4.0,삼성전자우,75000.0,200.0,-0.27%,100.0,617165.0,822887.0,77.73,1586008.0,19.53,NaN,NaN


이 중 마지막 열인 토론실은 필요 없는 열이며, 첫 번째 행과 같이 아무런 정보가 없는 행도 있습니다. 이를 다음과 같이 정리해줍니다.

In [20]:
table = table.drop(columns = '토론실')
table = table.dropna(subset = ['종목명'])

table.head()

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE
1,1.0,삼성전자,83600.0,1100.0,-1.30%,100.0,4990738.0,5969783.0,54.83,17791122.0,21.77,9.99
2,2.0,SK하이닉스,140000.0,4000.0,-2.78%,5000.0,1019203.0,728002.0,50.53,3247389.0,21.43,9.53
3,3.0,NAVER,383500.0,2000.0,+0.52%,100.0,629950.0,164263.0,57.07,463512.0,62.90,15.22
4,4.0,삼성전자우,75000.0,200.0,-0.27%,100.0,617165.0,822887.0,77.73,1586008.0,19.53,NaN
5,5.0,LG화학,812000.0,2000.0,+0.25%,5000.0,573210.0,70592.0,44.04,222739.0,123.99,2.93


1. 토론실 열을 제거합니다.
2. 종목명이 NA인 행은 삭제합니다.

이제 필요한 정보는 6자리 티커입니다. 티커 역시 개발자 도구 화면을 통해 어디에 위치하고 있음을 알고 있습니다. 티커를 추출하는 코드는 다음과 같습니다.

In [21]:
down_html = BeautifulSoup(down_table.content, 'html.parser')
down_link = down_html.select('tbody > tr > td > a')
symbol = [i.get('href') for i in down_link]

symbol[:10]

['/item/main.nhn?code=005930',
 '/item/board.nhn?code=005930',
 '/item/main.nhn?code=000660',
 '/item/board.nhn?code=000660',
 '/item/main.nhn?code=035420',
 '/item/board.nhn?code=035420',
 '/item/main.nhn?code=005935',
 '/item/board.nhn?code=005935',
 '/item/main.nhn?code=051910',
 '/item/board.nhn?code=051910']

1. `BeautifulSoup()` 함수를 통해 HTML 정보를 읽어옵니다.
2. `select()` 함수를 통해 찾고자 하는 태그를 입력합니다.
3. `for loop` 구분 내 `get()` 함수를 통해 href 속성에 해당 하는 값들만 찾습니다.

이를 통해 symbol에는 href 속성에 해당하는 링크 주소들이 저장됩니다. 이 중 마지막 6자리 글자만 추출하는 코드는 다음과 같습니다.

In [22]:
symbol_parse = [i.split('=')[-1] for i in symbol]

symbol_parse[:10]

['005930',
 '005930',
 '000660',
 '000660',
 '035420',
 '035420',
 '005935',
 '005935',
 '051910',
 '051910']

`for loop` 구문 내에서 `split()` 함수를 통해 문자를 나눠준 후, 마지막에 해당하는 부분만 선택합니다. 

결과를 살펴보면 티커에 해당하는 마지막 6글자만 추출되지만 동일한 내용이 두 번 연속해 추출됩니다. 이는 main.nhn?code=에 해당하는 부분은 종목명에 설정된 링크, board.nhn?code=에 해당하는 부분은 토론실에 설정된 링크이기 때문입니다.

In [23]:
symbol_parse = pd.unique(symbol_parse).tolist()

symbol_parse[:10]

['005930',
 '000660',
 '035420',
 '005935',
 '051910',
 '207940',
 '035720',
 '005380',
 '006400',
 '068270']

`unique()` 함수를 이용해 중복되는 티커를 제거하면 우리가 원하는 티커 부분만 깔끔하게 정리됩니다. 해당 내용을 위에서 구한 테이블에 입력한 후 데이터를 다듬는 과정은 다음과 같습니다.

In [24]:
table['N'] = symbol_parse
table = table.rename(columns = {'N':'종목코드'})
table = table.reset_index(drop = True)
ticker[j] = table

1. 위에서 구한 티커를 'N'열에 입력합니다.
2. 해당 열 이름을 종목코드로 변경합니다.
3. `dropna()` 함수를 통해 특정 행을 삭제했으므로, 행 이름을 초기화해줍니다.
4. ticker 딕셔너리에 정리된 데이터를 입력합니다.

위의 코드에서 i와 j 값을 for loop 구문에 이용하면 코스피와 코스닥 전 종목의 티커가 정리된 테이블을 만들 수 있습니다. 이를 전체 코드로 나타내면 다음과 같으며, 무한 크롤링 방지를 위해 각 loop 사이에 0.5초의 타임슬립을 적용하였습니다.

In [44]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

data = {}
mkt = [0, 1] # 0 은 코스피, i = 1 은 코스닥 종목

for i in mkt:
    
    ticker = {}
    url = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='+str(i)+'&page=1'
    down_table = rq.get(url)
    
    # 최종 페이지 번호 찾아주기
    down_html = BeautifulSoup(down_table.content, 'html.parser')
    navi_final = down_html.select_one('td.pgRR > a').get('href')
    
    navi_final_number = navi_final.split('=')[-1]
    navi_final_number = int(navi_final_number)    
    
    # 첫번째 부터 마지막 페이지까지 for loop를 이용하여 테이블 추출하기
    for j in tqdm(range(1, navi_final_number + 1)) :
        
        url_page = 'https://finance.naver.com/sise/sise_market_sum.nhn?sosok='+str(i)+"&page="+str(j)
        read_table = pd.read_html(url_page, encoding = "EUC-KR")
        table = read_table[1]
        
        table = table.drop(columns = '토론실') # 토론실 부분 삭제
        table = table.dropna(subset = ['종목명']) # 빈 행 삭제
        
        # 6자리 티커만 추출
        down_page = rq.get(url_page)        
        down_html = BeautifulSoup(down_page.content, 'html.parser')
        down_link = down_html.select('tbody > tr > td > a')
        symbol = [s.get('href') for s in down_link]
        
        symbol_parse = [p.split('=')[-1] for p in symbol]
        symbol_parse = pd.unique(symbol_parse).tolist()
        
        # 테이블에 티커 넣어준 후, 테이블 정리
        table['N'] = symbol_parse
        table = table.rename(columns = {'N':'종목코드'})
        table = table.reset_index(drop = True)
        ticker[j] = table    
        
        # 타임슬립 적용
        time.sleep(0.5)
        
    # 딕셔너리를 데이터프레임으로 만들기
    tickers = pd.concat(ticker.values())
    data[i] = tickers

# 코스피와 코스닥 데이터 묶기        
data_final = pd.concat(data.values())        
data_final = data_final.reset_index(drop = True)    

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:53<00:00,  1.80s/it]
